<a href="https://colab.research.google.com/github/joedockrill/heroku/blob/master/ClownClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clown Classifier**

So this is my homework from lesson 2 of the fastai course. It's a resnet50 model to classify clowns into "nice" or "scary".

I've provided some URLs at the bottom if you don't want to go and find your own. I had no expectations about the model dealing even remotely well with people who aren't clowns (although I could easily have added a third class for normal people), but I found it amusing the way it classified the pictures I gave it. You may not...

If the interface hasn't loaded below it's because it's still loading the model across from Google Drive, it can take a few seconds.

In [42]:
# classifier
import urllib.request
from fastai.vision import *

defaults.device = torch.device('cpu')

# honky looking model loading code gets called from a background thread now so Voila
# doesn't block the UI from rendering while the model downloads. bit prettier looking...
def download_model():
  MODEL_URL = "https://drive.google.com/uc?export=download&id=1sOChO7_ia07_4PIueLInj9VUj-_43nGu"
  urllib.request.urlretrieve(MODEL_URL, "model.pkl")

def classify(url):
  # getting round some kind of threading shenanigans by faking a static variable, shhhhh
  if("learner" not in classify.__dict__):
    classify.learner = load_learner(Path("."), "model.pkl")

  urllib.request.urlretrieve(url, "image.jpg")
  img = open_image("image.jpg")

  pred_class,_,probs = classify.learner.predict(img)
  return pred_class.obj, classify.learner.data.classes, probs.tolist()

In [47]:
# interface
import ipywidgets as widgets
from IPython.display import display

def classify_on_click(btn):
  htmlResult.value = "<h3>Analysing...</h3>"
  display_image(txtURL.value)
  pred_class,classes,probs = classify(txtURL.value)
  pred_class = pred_class.replace("-clown", "") #because i should have named my classes better...

  html = "<div align='center'>"
  html += "<h3>This clown is " + pred_class + "</h3>"
  html += "Probilities: " 
  html += classes[0] + "=" + "{:1.2f}".format(probs[0]) + ", "
  html += classes[1] + "=" + "{:1.2f}".format(probs[1]) 
  html += "</div>"
  htmlResult.value = html

def display_image(url):
  htmlImage.value= "<img src='" + url + "' width='300'>"

DEFAULT_IMG = "https://drive.google.com/uc?export=download&id=157sDYNJ0Z1zGf9rv6klbYnYig5elGmpF"

htmlImage = widgets.HTML(value="")
display_image(DEFAULT_IMG)

txtURL = widgets.Text(description="URL:", placeholder="Enter the URL of a clown picture")
txtURL.layout.width = "500px"
btnClassify = widgets.Button(description="Classify", disabled=True)
btnClassify.on_click(classify_on_click)
boxH = widgets.HBox([txtURL, btnClassify])

htmlResult = widgets.HTML(value="<h3>I fed clowns to a fastai convnet. They tasted funny.</h3>")
lblStatus = widgets.Label(value="Status: Loading")
lblStatus.layout.margin = "0px 0px 40px 0px"

vbox = widgets.VBox([htmlImage, boxH, htmlResult, lblStatus])
vbox.layout.align_items = "center"
display(vbox)

# give them some examples to play with without getting urls
def example_on_click(btn):
  txtURL.value = btn.tag
  classify_on_click(btnClassify)

def create_example(url):
  img = widgets.HTML("<img src='" + url + "' height='100'>")
  btn = widgets.Button(description="Test")
  btn.tag = url
  btn.on_click(example_on_click)
  vbox = widgets.VBox([img,btn])
  vbox.layout.align_items = "center"
  return vbox

urls = [
  "https://image.shutterstock.com/image-photo/funny-kid-clown-red-hair-260nw-1088492591.jpg", 
  "https://cdn.vox-cdn.com/thumbor/hySCY2Pd1Lg-k-Zlyo5xkrPBX20=/1400x0/filters:no_upscale()/cdn.vox-cdn.com/uploads/chorus_asset/file/12486813/Clown_8.0.jpg",
  "https://cdn.vox-cdn.com/thumbor/jUePyKTmAq2Z464lXYhpvUhuZ8I=/1400x0/filters:no_upscale()/cdn.vox-cdn.com/uploads/chorus_asset/file/12486807/Clown_3.0.jpg",
  "https://i.pinimg.com/originals/3b/be/45/3bbe455102303d77c8fc046b18679935.jpg", 
  "https://pyxis.nymag.com/v1/imgs/12f/0c0/c381b0aa9a68758e0f99d7df8ac75ad0f7-17-clowns-pennywise.rsquare.w700.jpg",
  "https://images-na.ssl-images-amazon.com/images/I/81HIWS0G1pL._SL1500_.jpg",
  "https://upload.wikimedia.org/wikipedia/commons/b/b1/Boris_Johnson_election_infobox.jpg", 
  "https://static01.nyt.com/newsgraphics/2019/08/01/candidate-pages/7d63f01f112e79da7ac60c0448a4047a155ff410/trump.jpg", 
  "https://www.etonline.com/sites/default/files/styles/max_970x546/public/images/2019-03/jimmy_fallon_gettyimages-1126646661_1280.jpg?h=c673cd1c&itok=Bj77ku-B"
  ]
  
items = []
for url in urls:
    items.append(create_example(url))

grid = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 33%)"))

# now load the model across from google drive on a background thread.
# ugly as sin but Voila blocks the UI from rendering until everything in the foreground has
# finished executing. doesn't work from a threading.Thread (starts but doesn't complete) but
# seems perfectly happy from concurrent.futures, go figure.

def async_load_model():
  lblStatus.value = "Downloading model from Google Drive..."
  download_model()
  display(grid)
  btnClassify.disabled = False
  lblStatus.value = ""
  
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
  future = executor.submit(async_load_model)

GridBox(children=(VBox(children=(HTML(value="<img src='https://image.shutterstock.com/image-photo/funny-kid-cl…

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
